In [162]:
! python3 -m spacy download en
! python3 -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [163]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
from tqdm import tqdm

In [164]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [165]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [166]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [167]:
SRC = Field(tokenize = tokenize_en, init_token = '<sos>', eos_token = '<eos>', lower = True, include_lengths = True)
TRG = Field(tokenize = tokenize_de, init_token = '<sos>', eos_token = '<eos>', lower = True)

In [168]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

In [169]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [170]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [171]:
BATCH_SIZE = 128
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort = lambda x : len(x.src),
    batch_size = BATCH_SIZE,
    device = device
)

In [172]:
batch = next(iter(train_iterator))

In [173]:
print(type(batch))
batch.__dict__

<class 'torchtext.data.batch.Batch'>


{'batch_size': 128,
 'dataset': <torchtext.datasets.translation.Multi30k at 0x7f4de56815f8>,
 'fields': dict_keys(['src', 'trg']),
 'input_fields': ['src', 'trg'],
 'src': (tensor([[   2,    2,    2,  ...,    2,    2,    2],
          [  18,    5,    5,  ...,   56,   78,    8],
          [ 126,   73,   34,  ...,  115, 2387, 2527],
          ...,
          [   4,    4,   65,  ...,    1,    1,    1],
          [   3,    3,    4,  ...,    1,    1,    1],
          [   1,    1,    3,  ...,    1,    1,    1]], device='cuda:0'),
  tensor([8, 8, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7,
          7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
          7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
          7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6,
          6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
          7, 7, 7, 6, 6, 6, 5, 4], device='cuda:0')),
 'targe

In [174]:
z = batch.src

In [175]:
class Encoder(nn.Module):
    def __init__(self, inp_dim, emb_dim, hid_dim, out_dim, dropout = 0.0):
        super().__init__()
        self.embedding = nn.Embedding(inp_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim, bidirectional = True)
        self.fc = nn.Linear(hid_dim * 2, out_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src, src_len):
        
        embedded = self.dropout(self.embedding(src))
        # embedded = [src_len, batch_size, emb_dim]
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.cpu(), enforce_sorted = False)

        packed_outputs, hidden = self.rnn(packed_embedded)
        # output = [src_len, batch_size, emb_dim * num_dir]
        # hidden = [num_dir * num layers, batch_size, emb_dim]

        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim = 1)))
        # hidden = [batch_size, num_dir * num_layers * emb_dim]
        # print('enc after concat hidden', hidden.shape)

        return outputs, hidden  

In [176]:
input_dim = 32
emb_dim = 256
enc_hid_dim = 512
dec_hid_dim = 512
dropout = 0.2

In [177]:
enc = Encoder(input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout)

In [178]:
# outputs, hidden = enc(torch.zeros(12, 32).to(torch.int64))

In [179]:
# outputs.shape, hidden.shape

In [180]:
# hidden = hidden.unsqueeze(0).repeat(12, 1, 1)

In [181]:
# hidden.shape

In [182]:
# hidden = hidden.repeat(12, 1, 1)
# hidden.shape

In [183]:
# atn = Attention(enc_hid_dim, dec_hid_dim)

In [184]:
# energy = torch.tanh(atn.attn(torch.cat((hidden, outputs), dim = 2)))
# energy.shape
# atten = atn.v(energy)
# atten.shape
# x = F.softmax(atten.squeeze(2), dim = 1)
# x.shape

In [185]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)

    def forward(self, encoder_outputs, hidden, mask):
        # encoder_outputs = [src_len, batchsize, enc_hid_dim * num dire]
        # hidden = [batch_size, dec_hid_dim]
        src_len = encoder_outputs.shape[0]
        
        # print(hidden.shape, encoder_outputs.shape)
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # hidden = [batch size, src len, dec hid dim]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder outputs = [batch size, src len, enc hid dim * 2]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))
        # energy = [batch size, src len, dec hid dim]
        attention = self.v(energy).squeeze(2)
        # attention = [batch_size, src len]
        attention = attention.masked_fill(mask == 0, -1e10)
        return F.softmax(attention, dim = 1)
        

In [186]:
class Decoder(nn.Module):
    def __init__(self, out_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = out_dim
        self.attention = attention
        self.embedding = nn.Embedding(out_dim, emb_dim)
        self.rnn = nn.GRU(enc_hid_dim * 2 + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear(enc_hid_dim * 2 + emb_dim + dec_hid_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, inp, encoder_outputs, hidden, mask):
        # input [batchsize]
        # encoder_outputs = [src_len, batchsize, enc_hid_dim * num dir]
        # hidden = [batch size, dec hid dim]
        # mask = [batch size, src len]
        input = inp.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(inp))
        # embedded = [batchsize, emb dim]
        embedded = embedded.unsqueeze(0)
        # embedded = [1, batchsize, emb dim]

        a = self.attention(encoder_outputs, hidden, mask)
        # [batch size, src len]
        a = a.unsqueeze(1)
        # a = [batch size, 1, src len]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # enc outputs = [batch size, src len, enc hid dim * num dir]
        weighted = torch.bmm(a, encoder_outputs)
        # weighted = [batch size, 1, enc hid dim * num dir]
        weighted = weighted.permute(1, 0, 2)
        # weighted = [1, batch size, enc hid dim * num dir]
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        # rnn_input = [1, batch size, enc hid dim *  + emb dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))

        #output = [seq len, batch size, dec hid dim * num dir]
        #hidden = [num layers * num dir, batch size, dec hid dim]

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        # prediction = [batch size, output dim]
        return prediction, hidden.squeeze(0)



In [187]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, src_pad_idx):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.src_pad_idx = src_pad_idx
    
    def create_mask(self, src):
        # print(src.shape)
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        # src, src_len = src
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        # print('src shape', src.shape)
        encoder_outputs, hidden = self.encoder(src, src_len)
        # print('encoder outputs hiddem', encoder_outputs.shape, hidden.shape)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        mask = self.create_mask(src)
        for t in range(1, trg_len):
            # print(t)
            # mask = self.create_mask(src_pad_idx)
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, encoder_outputs, hidden, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [188]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
src_pad_idx = SRC.vocab.stoi[SRC.pad_token]


attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device, src_pad_idx).to(device)

In [189]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7873, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5923, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=5923, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [190]:
optimizer = optim.Adam(model.parameters())

In [191]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [192]:
def train(model, train_iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(train_iterator):
        src, src_len = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, src_len, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(train_iterator)

In [193]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_len = batch.src
            trg = batch.trg

            output = model(src, src_len, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [194]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [195]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 26s
	Train Loss: 4.503 | Train PPL:  90.326
	 Val. Loss: 5.121 |  Val. PPL: 167.483
Epoch: 02 | Time: 0m 26s
	Train Loss: 3.331 | Train PPL:  27.959
	 Val. Loss: 4.242 |  Val. PPL:  69.563
Epoch: 03 | Time: 0m 26s
	Train Loss: 2.782 | Train PPL:  16.152
	 Val. Loss: 4.062 |  Val. PPL:  58.110
Epoch: 04 | Time: 0m 26s
	Train Loss: 2.470 | Train PPL:  11.822
	 Val. Loss: 3.922 |  Val. PPL:  50.505
Epoch: 05 | Time: 0m 26s
	Train Loss: 2.190 | Train PPL:   8.938
	 Val. Loss: 3.813 |  Val. PPL:  45.287
Epoch: 06 | Time: 0m 26s
	Train Loss: 1.964 | Train PPL:   7.128
	 Val. Loss: 3.782 |  Val. PPL:  43.887
Epoch: 07 | Time: 0m 26s
	Train Loss: 1.769 | Train PPL:   5.864
	 Val. Loss: 3.710 |  Val. PPL:  40.846
Epoch: 08 | Time: 0m 26s
	Train Loss: 1.600 | Train PPL:   4.954
	 Val. Loss: 3.669 |  Val. PPL:  39.229
Epoch: 09 | Time: 0m 26s
	Train Loss: 1.456 | Train PPL:   4.287
	 Val. Loss: 3.717 |  Val. PPL:  41.147
Epoch: 10 | Time: 0m 26s
	Train Loss: 1.363 | Train PPL